In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from scipy.optimize import minimize
test = sio.loadmat('test1.mat')
X = test['X']
y = test['y'].ravel()
weights = sio.loadmat('test2.mat')
theta1 = weights['Theta1']
theta2 = weights['Theta2']
import pdb

# 1 Neural Networks

In [36]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoidGrad(x):
    return sigmoid(x)*(1-sigmoid(x))

# 2 Forward and Backward Propagation

In [36]:
def nnCostFunction(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lamb):
    m = len(y)
    pdb.set_trace()
    theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))
    theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size + 1):],
                        (num_labels, (hidden_layer_size + 1)))
    z2 = theta1.dot(X.T)
    # z2=np.concatenate((np.ones((1,z2.shape[1])),z2),0)
    a2 = sigmoid(z2)
    a2 = np.concatenate((np.ones((1, a2.shape[1])), a2), 0)
    z3 = theta2.dot(a2)
    a3 = sigmoid(z3)
    posXMask = np.ravel_multi_index((y-1, np.arange(a3.shape[1])), a3.shape)
    a_3 = a3.ravel().copy()
    a__3 = a_3[np.ravel_multi_index((y-1, np.arange(a3.shape[1])), a3.shape)]
    a_3[posXMask] = np.zeros((len(a__3),))
    temp = np.zeros(a3.shape)
    temp[np.unravel_index(posXMask, a3.shape)] = np.ones((len(posXMask,)))
    del3 = a3-temp
    del2 = theta2.T[1:, :].dot(del3)*sigmoidGrad(z2)
    # del2=del2[:,1:]
    # pdb.set_trace()
    Del2 = 1./m*del3.dot(a2.T)
    Del2[:, 1:] = Del2[:, 1:]+lamb/m*theta2[:, 1:]
    Del1 = 1./m*del2.dot(X)
    Del1[:, 1:] = Del1[:, 1:]+lamb/m*theta1[:, 1:]
    return [-1./m*(np.sum(np.log(a__3))+np.sum(np.log(1-a_3))) 
            + lamb/2./m*(np.sum(theta1[:, 1:]**2)+np.sum(theta2[:, 1:]**2)),
            np.append(Del1.ravel(), Del2.ravel())]

In [33]:
epsilon_init = 0.12
theta_in1 = np.random.uniform(-epsilon_init, epsilon_init, theta1.shape)
theta_in2 = np.random.uniform(-epsilon_init, epsilon_init, theta2.shape)
X_ = np.concatenate((np.ones((X.shape[0], 1)), X), 1)

# Gradient Checking

In [26]:
from numpy.linalg import norm


def numGrad(cost,
            nn_params,
            input_layer_size,
            hidden_layer_size,
            num_labels,
            X, y, lamb):
    eps = 1e-6
    numG = np.zeros(nn_params.shape)
    costt = cost(nn_params,
                 input_layer_size,
                 hidden_layer_size,
                 num_labels,
                 X, y, lamb)
    for i in range(len(nn_params)):
        temp = np.zeros(nn_params.shape)
        temp[i] = eps
        # pdb.set_trace()
        params = nn_params+temp
        cost2 = cost(params,
                     input_layer_size,
                     hidden_layer_size,
                     num_labels,
                     X, y, lamb)[0]
        temp[i] = -eps
        params = nn_params+temp
        cost1 = cost(params,
                     input_layer_size,
                     hidden_layer_size,
                     num_labels,
                     X, y, lamb)[0]
        numG[i] = cost2-cost1

    numG = numG/2/eps
    pdb.set_trace()
    return norm(numG-costt[1])/norm(numG+costt[1])

In [32]:
theta_1 = np.random.uniform(-epsilon_init, epsilon_init, (3, 10))
theta_2 = np.random.uniform(-epsilon_init, epsilon_init, (10, 4))
theta_in1 = np.zeros((3, 10))
theta_in2 = np.zeros((10, 4))
numGrad(nnCostFunction,
        np.append(theta_1.ravel(), theta_2.ravel()),
        9, 3, 10, X_[30:50, :10], y[30:50], 1.)

> <ipython-input-26-18ad0927b04d>(36)numGrad()
-> return norm(numG-costt[1])/norm(numG+costt[1])
(Pdb) c


5.9293170235728872e-10

In [ ]:
minimize(nnCostFunction, np.append(theta_1.ravel(), theta_2.ravel()),         args=(
    400, 25, 10, X_, y, 1.), method='BFGS', jac=True,         options={'gtol': 1e-6, 'disp': True})

> <ipython-input-36-c40482f540c6>(12)nnCostFunction()
-> theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
(Pdb) nn_params.shape
(70L,)


In [9]:

nnCostFunction(np.append(theta1.ravel(), theta2.ravel()),
               400,
               25,
               10,
               X_, y, 1.)

[0.38376985909092365,
 array([  1.67404866e-02,  -2.11248326e-12,   4.38829369e-13, ...,
         -8.89348084e-02,  -8.44931218e-02,  -1.17334401e-02])]